# Camada Bronze

Ingestão dos dados brutos baixados do Banco Central e armazenamento em formato Delta Lake (bronze_pix), mantendo a estrutura original. Esta camada serve como nossa fonte primária e é uma cópia fiel dos dados de origem.

Comecei criando uma base chamada bronze dentro do workspace e fiz upload nela do arquivo csv.

In [0]:
SHOW SCHEMAS;


databaseName
analytics
bronze
default
gold
information_schema
silver


Criar o Schema Bronze (se ainda não existir)

In [0]:
CREATE SCHEMA IF NOT EXISTS bronze;

In [0]:
USE SCHEMA bronze;


Criei a TABELA BRONZE apontando para o Volume

In [0]:
CREATE TABLE IF NOT EXISTS bronze.transacoes_pix AS
SELECT *
FROM read_files(
  '/Volumes/workspace/bronze/bronze_pix/',
  format => 'csv',
  header => true,
  inferSchema => true
);


num_affected_rows,num_inserted_rows


In [0]:
DESCRIBE TABLE bronze.transacoes_pix;



col_name,data_type,comment
AnoMes,int,null
PAG_PFPJ,string,null
REC_PFPJ,string,null
PAG_REGIAO,string,null
REC_REGIAO,string,null
PAG_IDADE,string,null
REC_IDADE,string,null
FORMAINICIACAO,string,null
NATUREZA,string,null
FINALIDADE,string,null


Fiz uma contagem para verificar o número de linhas.

In [0]:
SELECT COUNT(*) FROM bronze.transacoes_pix;


count(1)
598924


In [0]:
SELECT current_schema();


current_schema()
bronze


Durante a ingestão dos dados na camada Bronze, foi identificada a presença da coluna _rescued_data, pesquisando melhor entendi que ela é gerada automaticamente pelo Databricks como mecanismo de tolerância a variações de schema. Essa coluna foi mantida na camada Bronze para preservação dos dados originais e descartada na camada Silver, onde o schema passa a ser controlado.
Para garantir confiabilidade fiz uma verificação quando ao número de linhas  da coluna _rescued_data.

In [0]:
SELECT
  COUNT(*) AS total_linhas,
  COUNT(_rescued_data) AS linhas_com_rescue
FROM bronze.transacoes_pix;

total_linhas,linhas_com_rescue
598924,0


Como a quantidade de linhas é igual a zero, não há problema real nos dados.

## Resumo

Os dados brutos foram inicialmente armazenados em um Volume gerenciado pelo Catalog. Em seguida, foi criada uma tabela Bronze no Databricks, associada diretamente ao local físico de armazenamento, permitindo a consulta dos dados via SQL de forma estruturada.

A Tabela Bronze transacoes_pix contém os dados brutos das transações PIX, conforme disponibilizado pelo Banco Central do Brasil.
A estrutura contempla atributos temporais, características do pagador e do recebedor, bem como métricas agregadas de valor e quantidade.

| Coluna         | Descrição                   |
| -------------- | --------------------------- |
| AnoMes         | Ano e mês de referência     |
| PAG_PFPJ       | Tipo de pessoa do pagador   |
| REC_PFPJ       | Tipo de pessoa do recebedor |
| PAG_REGIAO     | Região do pagador           |
| REC_REGIAO     | Região do recebedor         |
| PAG_IDADE      | Idade do pagador            |
| REC_IDADE      | Idade do recebedor          |
| FORMAINICIACAO | Forma de iniciação          |
| NATUREZA       | Natureza da transação       |
| FINALIDADE     | Finalidade                  |
| VALOR          | Valor total das transações  |
| QUANTIDADE     | Quantidade de transações    |
